In [1]:
from elasticsearch import Elasticsearch
from datetime import datetime

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [408]:
I = json.dumps({'status': 'idle', 'origin': None, 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 1, 'name': 'driver_1', 'p2': None, 'p1': None, 'ctime': '2016-09-30 23:36:03.753770', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.7778, -73.9448]})
I = json.loads(I)
ctime = I['ctime']
driver = I ['id']
status = I ['status']
location = I['location']
name = I ["name"]

try:
    tmp = datetime.strptime("{}".format(ctime), '%Y-%m-%d %H:%M:%S.%f')
    ctime = tmp.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
except:
    pass


In [409]:
geo_query = { "from" : 0, "size" : 1,
                      "query": {"filtered": { "filter": { "geo_distance": \
                          { "location": location,  "distance_type": "plane", "distance": "100km" }}, "query": { "query_string": { "query": "wait" }}}}}

res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])

if len(res['hits']['hits'])>0: 
    passenger = res['hits']['hits'][0]["_source"]
    doc = json.dumps({"status": "pickup", "driver": driver})
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='passenger', doc_type='rolling', id=passenger['id'], body=q, ignore=[400,404])
    doc = {"status": "pickup", "ctime": ctime, \
           "location": location, 'name': name, \
           'destination':  passenger['location'], \
           'destinationid': passenger['id']}
    if not p1:
        doc['p1'] = passenger['id']
    elif not p2:
        doc['p2'] = passenger['id']
    else:
        pass # It's full!
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    doc = {"status": "idle", "ctime": ctime, "location": location, 'name': name}
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    print doc
else:
    doc = {"ctime": ctime, "location": location}
    doc = json.dumps(doc)
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                        body=q)
    print(doc)

{"ctime": "2016-09-30T23:36:03.753770Z", "location": [40.7778, -73.9448]}


In [8]:
window = 'now-2h'


q = {'size': 0,
    "aggs" : {
                    "filter" : { "term": { "status": "arrived" } },

        "avg_ptime" : { "avg" : { "field" : "ptime" } }
    }
}

q = {
  "query": {
    "exists": {"field": {"match"}}
  },
  "size": 1,
  "sort": [
    {
      "_timestamp": {
        "order": "desc"
      }
    }
  ]
}

q = { 'size': 0,
        "aggs" : {
            "avg_wait" : { "filter" : {  "range" : { "ctime" : { "gt" : window,}}},
            "aggs" : {
                "avg_wait" : { "avg" : { "field" : "ptime" } }
                }}}}

res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])

In [10]:
res

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'avg_wait': {u'avg_wait': {u'value': 560.8333333333334},
   u'doc_count': 194}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 194},
 u'timed_out': False,
 u'took': 1}

In [6]:
res['aggregations']['avg_wait']['avg_wait']['value']

0.0